# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [20]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [21]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [22]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [24]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [26]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

####❓ Answer #1:

Qdrant uses several techniques to ensure high performance:

- Hierarchical Navigable Small World
    - this graph-based indexing algorithm implements Approximate Nearest Neighbors by building a multi-layer navigation structure, enabling fast and efficient searches

- Compression Techniques
    - Product Quantization
    - Scalar Quantization
    - Binary Quantization
    these techniques reduce storage requirements and improve search efficiency by transforming high-dimensional vectors into new, less precise representations with minimal loss in accuracy.

- Multi-Vector Support
    - Qdrant allows multiple vectors per document, enhancing search results by combining different aspects of the data, like the title and body of a document.

- Memory or On-Disk Storage
    - users can choose to store vectors in memory for speed or on-disk for greater storage capacity, providing flexibility based on their needs.

- Distributed Deployment
    - this allows data to be distributed across multiple peers, extending storage capabilities and increasing system stability.

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [27]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [28]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [29]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '3c088ada53914441941b841b64dba4b5', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [30]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [31]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [32]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [33]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

####🏗️ Activity sollution #1:
Capture Question
    - user's question is taken
Retrieve Context
    - find similar documents using the question
Prepare Inputs
- pass question unchanged, add context
Format Prompt
- Combine question and context
Generate Answer
- LLM processes the prompt and produces a response


Let's test it out!

In [34]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

A rule of thumb for selecting an industry to invest in is to get right to the center of it.


In [35]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '5a413c7aa8aa495a9b6611fff78f213e', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [36]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

####❓ Answer #2:

Splitting documents differently is crucial to avoid overfitting and ensure diverse evaluation. If we use the same chunks for training and testing, the RAG (Retrieval-Augmented Generation) system won't be properly tested. Large Language Models often follow common patterns, so varying the chunks helps create more realistic and challenging test sets. This diversity ensures the RAG pipeline is robust and performs well in real-world scenarios, providing a true measure of its effectiveness.

In [37]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [41]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 2, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/2 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How should emails relating to current projects...,[you can reply to a lot of messages with “I’m ...,Emails relating to current projects or pressin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does hiring highly intelligent individuals...,"[relation between raw intelligence, as measure...",Hiring highly intelligent individuals can pote...,multi_context,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


after multiple times trying to solve the limit (tier 1 and cant be upgraded :() i decided to reduce number of samples to 2 (i spent 32€, i hit the brick in later code and didnt save the notebook and needed to run it twice from begining )

####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

####❓ Answer #3:

The mapping {simple: 0.5, reasoning: 0.25, multi_context: 0.25} refers to the distribution of different types of questions in the synthetic test set generated by Ragas. 

detailed info:
- Simple (0.5)
    - 50% of the questions will be straightforward and easy to answer. These questions usually don't require complex reasoning or multiple sources of information.

- Reasoning (0.25)
    - 25% of the questions will involve reasoning. These questions are designed to test the model's ability to make logical inferences and connections.

- Multi-Context (0.25)
    - 25% of the questions will require information from multiple sections of the documents. This ensures the model can handle more complex queries that need broader context to answer accurately.

By using this distribution, the generated test set will comprehensively evaluate the model's performance across different types of questions, ensuring robustness and reliability in various scenarios​

Let's look at the output and see what we can learn about it!

In [42]:
testset.test_data[0]

DataRow(question='How should emails relating to current projects or pressing issues be handled in terms of replying to them?', contexts=['you can reply to a lot of messages with “I’m sorry, I’m not keep-\ning a schedule in 2007, I can’t commit to that.”\nThird, emails relating to topics that are current working pro-\njects or pressing issues go into temporary subfolders of a\nfolder called Action.\nYou should only have Action subfolders for the things that really\nmatter, right now.\nThose subfolders then get used, and the messages in them\nprocessed, when you are working on their respective projects in\nthe normal course of your day.\nFourth, aside from those temporary Action subfolders, only'], ground_truth='Emails relating to current projects or pressing issues should go into temporary subfolders of a folder called Action. These subfolders are used when working on their respective projects in the normal course of the day. Apart from these temporary subfolders, other emails should no

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [43]:
test_df = testset.to_pandas()

In [44]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How should emails relating to current projects...,[you can reply to a lot of messages with “I’m ...,Emails relating to current projects or pressin...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How does hiring highly intelligent individuals...,"[relation between raw intelligence, as measure...",Hiring highly intelligent individuals can pote...,multi_context,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [45]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [46]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [47]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [48]:
response_dataset[0]

{'question': 'How should emails relating to current projects or pressing issues be handled in terms of replying to them?',
 'answer': "I don't know.",
 'contexts': ['inbox that haven’t been dealt with, I Xnd it hard to concentrate\non other things.\nThe urge to go back to my email is nearly overpowering.\n(I am apparently seriously addicted to endorphins.)',
  'account just for them and leave that open all day, but keep\nyour primary email closed. And never give out the family email\naddress to anyone noncritical — including your boss.)',
  'matter, right now.\nThose subfolders then get used, and the messages in them\nprocessed, when you are working on their respective projects in\nthe normal course of your day.',
  'respond to all the emails from people who want to buy.\nPart 4: The only thing that matters\n27'],
 'ground_truth': 'Emails relating to current projects or pressing issues should go into temporary subfolders of a folder called Action. These subfolders are used when working

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [49]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [50]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

No statements were generated from the answer.


In [51]:
results

{'faithfulness': 0.5000, 'answer_relevancy': 0.0000, 'context_recall': 0.8333, 'context_precision': 0.7500, 'answer_correctness': 0.2728}

In [52]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How should emails relating to current projects...,I don't know.,"[inbox that haven’t been dealt with, I Xnd it ...",Emails relating to current projects or pressin...,NaN,0.0,0.666667,0.5,0.179280
1,How does hiring highly intelligent individuals...,I don't know.,"[Especially in this industry.\nYou will read, ...",Hiring highly intelligent individuals can pote...,0.5,0.0,1.000000,1.0,0.366399


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [53]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [54]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [55]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [56]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [57]:
print(response["answer"])

The provided context does not contain any information about Taylor Swift or any feuds she may be involved in.


In [58]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [59]:
print(response["answer"])

The context provided does not give specific details about a feud or why individuals might be feuding. It discusses human tendencies towards dislike and hate, the continuous history of war, and various factors that contribute to success or failure in business, but it does not mention a specific feud or the reasons behind it.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [60]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [61]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [62]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How should emails relating to current projects...,Emails relating to current projects or pressin...,"[inbox that haven’t been dealt with, I Xnd it ...",Emails relating to current projects or pressin...,0.75,0.981667,0.0,0.455397,0.218632
1,How does hiring highly intelligent individuals...,The impact of hiring highly intelligent indivi...,"[Especially in this industry.\nYou will read, ...",Hiring highly intelligent individuals can pote...,0.25,0.000000,1.0,1.000000,0.390568


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [64]:
results

{'faithfulness': 0.5000, 'answer_relevancy': 0.0000, 'context_recall': 0.8333, 'context_precision': 0.7500, 'answer_correctness': 0.2728}

And see how our advanced retrieval modified our chain!

In [65]:
advanced_retrieval_results

{'faithfulness': 0.5000, 'answer_relevancy': 0.4908, 'context_recall': 0.5000, 'context_precision': 0.7277, 'answer_correctness': 0.3046}

In [66]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.500000,0.500000,0.000000
1,answer_relevancy,0.000000,0.490833,0.490833
2,context_recall,0.833333,0.500000,-0.333333
3,context_precision,0.750000,0.727698,-0.022302
4,answer_correctness,0.272839,0.304600,0.031761


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

- call the embedding model that we will user

In [67]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

initailization of qdrant vector, loading document in memory

In [68]:
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

create a retreiver from vector store

In [69]:
new_retriever = vector_store.as_retriever()

advanced retriever (is able to get more diverse set of documents)

In [70]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

retreival chain that retrieves documents

In [71]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

for each test question, get the response from the retrieval chain. Save the answers and contexts in lists.

In [72]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Create a dataset from the new questions, answers, contexts, and ground truth.

In [73]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Evaluate the new results using the provided metrics and configuration.

In [74]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluate the new dataset using the specified metrics and configuration, and store the results.

In [75]:
new_advanced_retrieval_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.4742, 'context_recall': 0.5000, 'context_precision': 0.7411, 'answer_correctness': 0.4812}

Create dataframes for baseline, original, and new evaluation results. Merge these dataframes on the 'Metric' column and calculate the differences between various metrics to compare performance improvements.

In [76]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.500000,0.500000,1.000000,0.500000,0.500000
1,answer_relevancy,0.000000,0.490833,0.474202,-0.016631,0.474202
2,context_recall,0.833333,0.500000,0.500000,0.000000,-0.333333
3,context_precision,0.750000,0.727698,0.741071,0.013373,-0.008929
4,answer_correctness,0.272839,0.304600,0.481186,0.176586,0.208347


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

####❓ Answer #4:

After running the notebook several times, I observed varying results between the text-embedding-3-small (TE3 + MQR) model and the text-embedding-ada-002 (ADA + MQR) model. Here is a simplified comparison:

Faithfulness: TE3 + MQR shows a consistent 100% improvement compared to both ADA + MQR and ADA + Baseline, indicating more accurate outputs.

Answer Relevancy: Previously, TE3 + MQR had a slight decrease of 1.6% compared to ADA + MQR. This time, it's almost on par with ADA + MQR, with only a minor difference.

Context Recall: TE3 + MQR matches ADA + MQR but shows a decrease of 33.3% compared to ADA + Baseline, similar to previous runs.

Context Precision: TE3 + MQR shows a slight improvement of about 1.4% compared to ADA + MQR, consistent with previous results.

Answer Correctness: TE3 + MQR improved significantly by around 17.7% compared to ADA + MQR, similar to past observations.

Overall, despite some minor variations in specific metrics, TE3 + MQR consistently demonstrates significant improvements in key areas like faithfulness and answer correctness. Therefore, I believe the text-embedding-3-small model is significantly better than the ADA model in these respects.

## BONUS ACTIVITY: Using a Better Generator

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [82]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import context_precision, context_recall, answer_relevancy, faithfulness
import pandas as pd
from datasets import Dataset

# Initialize LLMs and embeddings
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

# Generate test set
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# Assume eval_documents is already defined and contains your evaluation documents
testset = generator.generate_with_langchain_docs(eval_documents, 2, distributions, is_async=False)
testset_df = testset.to_pandas()

# Extract test questions and ground truths
test_questions = testset_df['question'].tolist()
test_groundtruths = testset_df['ground_truth'].tolist()

# Initialize the primary LLM with a specified model and temperature
primary_qa_llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Assuming new_retriever and document_chain are already defined
# Create an advanced retriever using the new retriever and the primary LLM
advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

# Create a retrieval chain using the advanced retriever and document chain
retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

# Initialize lists to store answers and contexts
answers = []
contexts = []

# Process each test question through the retrieval chain
for question in test_questions:
    response = retrieval_chain.invoke({"input": question})
    answers.append(response["answer"])
    contexts.append([context.page_content for context in response["context"]])

# Create a dataset from the responses
response_dataset = Dataset.from_dict({
    "question": test_questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": test_groundtruths
})

# Define the metrics using the appropriate classes
metrics = [context_precision, context_recall, answer_relevancy, faithfulness]

# Evaluate the advanced retrieval results
advanced_retrieval_results = evaluate(response_dataset, metrics)

# Display evaluation results
print(advanced_retrieval_results)

# Format the results for visualization
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])
df_comparison2 = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR (GPT-4o)'])

# Merge the dataframes for comparison
df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")
df_merged = pd.merge(df_merged, df_comparison2, on="Metric")

# Calculate difference between TE3 + MQR and TE3 + MQR (GPT-4o)
df_merged['TE3 + MQR -> TE3 + MQR (GPT-4o)'] = df_merged['TE3 + MQR (GPT-4o)'] - df_merged['TE3 + MQR']

# Display the merged dataframe
print(df_merged)

# Compare a few answers with ground truth
for resp in response_dataset.shuffle().take(3):
    print('Q: ' + resp['question'] + '\n')
    print('A: ' + resp['answer'] + '\n')
    print('TRUTH: ' + resp['ground_truth'] + '\n')
    print('-----')


embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/site-packages/ragas/executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/site-packages/ragas/executor.py", line 83, in _aresults
    raise e
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/site-packages/ragas/executor.py", line 78, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/djuhas/anaconda3/envs/aie3week2d1/lib/python3.11/asyncio/tasks.py", line 605, in _wait_for_one


ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.